In [58]:
import pandas as pd
import numpy as np

In [59]:
alabama = 'data/alabama_results.xlsx'
arkansas = 'data/arkansas_results.xlsx'
california = 'data/california_results.xlsx'
colorado = 'data/colorado_results.xlsx'
iowa = 'data/iowa_results.xlsx'
minnesota = 'data/minnesota_results.xlsx'
nevada = 'data/nevada_results.xlsx'
new_hampshire = 'data/new_hampshire_results.xlsx'
north_carolina = 'data/north-carolina_results.xlsx'
oklahoma = 'data/oklahoma_results.xlsx'
south_carolina = 'data/south_carolina_results.xlsx'
tennesse = 'data/tennessee_results.xlsx'
texas = 'data/texas_results.xlsx'
utah = 'data/utah_results.xlsx'
virginia = 'data/virginia_results.xlsx'


In [150]:
results = pd.read_excel(virginia)

In [151]:
active_candidates = ['Bernie Sanders',
                       'Joe Biden']
active_candidates_LN = ['Sanders',
                       'Biden']

In [152]:
# results_df = results.drop('State-wide', axis=1)
# results_df = results_df.melt(id_vars=['Candidate'],value_vars=results_df.columns.to_list()[1:-1])

# results_df = results_df[~results_df['value'].isin(['—'])]

# results_df['State'] = "NH"

# results_df = results_df[results_df['Candidate'] != "Total"]

# results_df = results_df[results_df['Candidate'].isin(active_candidates)]


# results_df.rename(columns={"Candidate": "last name", "variable": "county", "value": "votes", "State": "state"}, inplace=True)
# results_df


In [153]:
### This code works for data from Washington Post
results_df = results.transpose().reset_index()
results_df.columns = results_df.iloc[0]
results_df = results_df.drop(results_df.index[0])
results_df = results_df.melt(id_vars=['Counties'],value_vars=results_df.columns.to_list()[1:-1])
results_df['State'] = "VA"
results_df = results_df[results_df['Counties'].isin(active_candidates_LN)]
results_df = results_df.sort_values(by=[0, 'value'], ascending=False)
iowa_counties = results_df[0].unique()

results_df.rename(columns={"Counties": "last name", 0: "county", "State": "state", 'value': 'votes'}, inplace=True)    

In [154]:
results_df

,last name,county,votes,state
1966,Biden,Wythe,656,VA
1965,Sanders,Wythe,424,VA
1951,Biden,Wise,552,VA
1950,Sanders,Wise,469,VA
1936,Biden,Winchester City,1666,VA
1935,Sanders,Winchester City,977,VA
1921,Biden,Williamsburg City,1434,VA
1920,Sanders,Williamsburg City,1131,VA
1906,Biden,Westmoreland,1438,VA
1905,Sanders,Westmoreland,345,VA


In [155]:
# all_results = results_df

In [157]:
all_results = all_results.append(results_df)

In [158]:
all_results

,last name,county,votes,state
976,Biden,Wilcox,1864,AL
975,Sanders,Wilcox,284,AL
961,Biden,Washington,1109,AL
960,Sanders,Washington,156,AL
946,Biden,Walker,1390,AL
945,Sanders,Walker,538,AL
931,Biden,Tuscaloosa,11825,AL
930,Sanders,Tuscaloosa,3552,AL
916,Biden,Tallapoosa,1762,AL
915,Sanders,Tallapoosa,302,AL


In [159]:
all_results.to_csv('combined_results.csv')